# Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

## Table of Contents
- [Part 1 - Data Scraping](#part-1)
- [Part 2 - Geocoding](#part-2)


<div id='part-1'/>

____
## Part 1 - Data Scraping

Input data [Wikipedia: List of postal codes of Canada: M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [1]:
from bs4 import BeautifulSoup
import urllib3.request
import pandas as pd
import numpy as np


- **Input data is obtained from Wikipedia via http request.**
- **_"BeatifulSoup"_ object is created.**

In [2]:
page_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# if you are behind a firewall set the proper url, including protocol, host and port.
#   (ex: http://internal-proxy:80)
proxy_url = ""

if proxy_url.strip() != "":
    # using proxy
    http = urllib3.ProxyManager(proxy_url)
else:
    # direct internet connection
    http = urllib3.PoolManager()

req = http.request('GET', page_url)
soup = BeautifulSoup(req.data, 'html.parser')


C:\ds\anaconda\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


  
- **HTML post codes table is parsed**
- **Rows with 'Not assigned' borough are dropped.**
- **Pandas dataframe is constructed.**
  

In [3]:
# locate postcode table
toronto_table = soup.find('table',{'class':'wikitable sortable'})

# process table rows
toronto_data = []
rows = toronto_table.findAll('tr')
for row in rows:
    row_items = row.findAll('td')
    if len(row_items) > 0:
        postcode = row_items[0].text.strip()
        borough = row_items[1].text.strip()
        if borough.lower() != "not assigned":
            neighborhood = row_items[2].text.strip()
            toronto_data.append((postcode, borough, neighborhood))

# create pandas dataframe with scrapped data
raw_df = pd.DataFrame(toronto_data, columns=['PostalCode', 'Borough', 'Neighborhood'])
raw_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


  
- **Combine neighborhoods belonging to the same borough in one row.**
- **Replace _'Not assigned'_ neighborhoods with Borougth's name.**
  

In [4]:
grouped = []
for name, group in raw_df.groupby(['PostalCode', 'Borough'])['Neighborhood']:
    nblist = ''.join(str(x) + ", " for x in group.tolist()).strip(", ")
    if nblist == "Not assigned":
        nblist = name[1]
    grouped.append((name[0], name[1], nblist))

postcode_df = pd.DataFrame(grouped, columns=['PostalCode', 'Borough', 'Neighborhood'])
postcode_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
# just for verification. This query should return no rows.
postcode_df.query("Neighborhood == 'Not assigned'")

,PostalCode,Borough,Neighborhood


In [6]:
# verify a known 'Not assigned' Neighborhood case, it should be equal to Borough. 
postcode_df.query("PostalCode == 'M7A'") 

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


  
- **Final assignament requirement: dataframe shape is shown.**
  

In [7]:
postcode_df.shape

(103, 3)

<div id='part-2'/>

___
## Part 2 : Geocoding

> Geocoder doesn't works for me, all the time I get 'None' as response.  
> Therefore I downloaded the 'Geospatial_Coordinates.csv' and got geocoding from that file.
  

In [8]:
import csv
with open('Geospatial_Coordinates.csv', 'rt') as geo_file:
    geo_reader = csv.reader(geo_file, delimiter=',')
    for row in geo_reader:
        #print(', '.join(row))
        postcode_df.loc[postcode_df['PostalCode'] == row[0], 'Latitude'] = row[1]        
        postcode_df.loc[postcode_df['PostalCode'] == row[0], 'Longitude'] = row[2]

In [9]:
postcode_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761


In [10]:
postcode_df.shape

(103, 5)